# DB2-Salesforce populate survey URLs obtain from DB2

In [1]:
# Parameters
# API settings
api_url = '/services/data/v43.0/sobjects'
external_id = 'nanoHUB_user_ID__c'
object_id = 'Contact'

# login parameters to be handled by Papermill
from nanoHUB.application import Application

application = Application.get_instance()
nanohub_db = application.new_db_engine('nanohub')

salesforce = application.new_salesforce_engine()
db_s = salesforce

nanoHUB - Serving Students, Researchers & Instructors


Obtained Salesforce access token ...... True


In [2]:
import pandas as pd
import datetime

## Get the URL code from DB2


In [3]:
# page_id indicates the question
url_code_df = pd.read_sql_query("select user_id, code, expiration from jos_reply_access_codes \
where page_id = 2 order by user_id desc;",nanohub_db) #limit 5000

new_pref_code_df = pd.read_sql_query("select user_id, code, expiration from jos_reply_access_codes \
where page_id = 1 order by user_id desc;",nanohub_db) # limit 5000

In [4]:
display(url_code_df.head(2))

,user_id,code,expiration
0,342930,bdc145cf177a99d7321679524e1cad79fabe7441e08b7e...,2021-12-31 23:59:59
1,342929,7ef8bcd7eb8c68c7d7917ce3f236f553c6d17fc7e44c4c...,2021-12-31 23:59:59


In [5]:
## populating full url codes
url_base = "https://nanohub.org/reply/pages/2?code="

from copy import deepcopy
codes = deepcopy(url_code_df['code'])

#import time
#st = time.time()
codes = codes.apply(lambda x: url_base+x)
#tt = st-time.time()
#print(tt)
print(codes[0])


https://nanohub.org/reply/pages/2?code=bdc145cf177a99d7321679524e1cad79fabe7441e08b7e40734a229c0fa44ada


In [6]:
## add the new code to the dataframe
url_code_df['code'] = codes
url_code_df = url_code_df.rename(columns={"user_id": "nanoHUB_user_ID__c", "code":"nh_use_survey_link__c"})
display(url_code_df.head(2))

,nanoHUB_user_ID__c,nh_use_survey_link__c,expiration
0,342930,https://nanohub.org/reply/pages/2?code=bdc145c...,2021-12-31 23:59:59
1,342929,https://nanohub.org/reply/pages/2?code=7ef8bcd...,2021-12-31 23:59:59


In [7]:
## determining the new url survey links
url_base1 = 'https://nanohub.org/reply/email-subscriptions?code='

codes = deepcopy(new_pref_code_df['code'])

codes = codes.apply(lambda x: url_base1+x)

print(codes[0])
new_pref_code_df['code'] = codes

https://nanohub.org/reply/email-subscriptions?code=433cc1b94e56b73c3df84280b9316d7471a91f6970891312641ab768173a37ec


In [8]:
display(new_pref_code_df.head(5))

,user_id,code,expiration
0,342930,https://nanohub.org/reply/email-subscriptions?...,2021-12-31 23:59:59
1,342929,https://nanohub.org/reply/email-subscriptions?...,2021-12-31 23:59:59
2,342928,https://nanohub.org/reply/email-subscriptions?...,2021-12-31 23:59:59
3,342927,https://nanohub.org/reply/email-subscriptions?...,2021-12-31 23:59:59
4,342926,https://nanohub.org/reply/email-subscriptions?...,2021-12-31 23:59:59


In [9]:
# expiration dates can be different, so do a left join instead of "url_code_df['new_preferences_survey_url__c'] = codes"
unif_df = pd.merge(url_code_df,new_pref_code_df,how='left',right_on='user_id',left_on='nanoHUB_user_ID__c')

display(unif_df.head(2))
display(unif_df.tail(2))

,nanoHUB_user_ID__c,nh_use_survey_link__c,expiration_x,user_id,code,expiration_y
0,342930,https://nanohub.org/reply/pages/2?code=bdc145c...,2021-12-31 23:59:59,342930,https://nanohub.org/reply/email-subscriptions?...,2021-12-31 23:59:59
1,342929,https://nanohub.org/reply/pages/2?code=7ef8bcd...,2021-12-31 23:59:59,342929,https://nanohub.org/reply/email-subscriptions?...,2021-12-31 23:59:59


,nanoHUB_user_ID__c,nh_use_survey_link__c,expiration_x,user_id,code,expiration_y
254206,1000,https://nanohub.org/reply/pages/2?code=ejjJycB...,2021-12-31 23:59:59,1000,https://nanohub.org/reply/email-subscriptions?...,2021-12-31 23:59:59
254207,998,https://nanohub.org/reply/pages/2?code=rWB03ln...,2021-12-31 23:59:59,998,https://nanohub.org/reply/email-subscriptions?...,2021-12-31 23:59:59


In [10]:
# some formatting for SF
unif_df = unif_df.rename(columns={'expiration_x':'nh_use_expire__c',\
                'code':'new_preferences_survey_url__c','expiration_y':'new_prefs_expire__c'})
unif_df = unif_df.drop(columns=['user_id']) #unified_dataframe

In [11]:
display(unif_df.head(2))
display(unif_df.tail(2))

,nanoHUB_user_ID__c,nh_use_survey_link__c,nh_use_expire__c,new_preferences_survey_url__c,new_prefs_expire__c
0,342930,https://nanohub.org/reply/pages/2?code=bdc145c...,2021-12-31 23:59:59,https://nanohub.org/reply/email-subscriptions?...,2021-12-31 23:59:59
1,342929,https://nanohub.org/reply/pages/2?code=7ef8bcd...,2021-12-31 23:59:59,https://nanohub.org/reply/email-subscriptions?...,2021-12-31 23:59:59


,nanoHUB_user_ID__c,nh_use_survey_link__c,nh_use_expire__c,new_preferences_survey_url__c,new_prefs_expire__c
254206,1000,https://nanohub.org/reply/pages/2?code=ejjJycB...,2021-12-31 23:59:59,https://nanohub.org/reply/email-subscriptions?...,2021-12-31 23:59:59
254207,998,https://nanohub.org/reply/pages/2?code=rWB03ln...,2021-12-31 23:59:59,https://nanohub.org/reply/email-subscriptions?...,2021-12-31 23:59:59


## Send to SF

In [12]:
db_s.object_id = object_id
db_s.external_id = external_id

# send data to Salesforce
db_s.send_data(unif_df)

[Success] Bulk job creation successful. Job ID = 7505w00000b1ooVAAQ
hello


[Success] CSV upload successful. Job ID = 7505w00000b1ooVAAQ


[Success] Closing job successful. Job ID = 7505w00000b1ooVAAQ


In [13]:
# check status
db_s.check_bulk_status()

{'id': '7505w00000b1ooVAAQ',
 'operation': 'upsert',
 'object': 'Contact',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2021-10-22T11:57:31.000+0000',
 'systemModstamp': '2021-10-22T11:59:06.000+0000',
 'state': 'UploadComplete',
 'externalIdFieldName': 'nanoHUB_user_ID__c',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 0,
 'numberRecordsFailed': 0,
 'retries': 0,
 'totalProcessingTime': 0,
 'apiActiveProcessingTime': 0,
 'apexProcessingTime': 0}

In [14]:
# check status
from pprint import pprint

pprint(db_s.check_bulk_failed_results())

''
